<a href="https://colab.research.google.com/github/ugur-sa/Bachelorarbeit/blob/main/SentimentTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# lies die Zeilen der datei links.txt aus und gib die länge der liste aus

with open("links.txt", 'r') as f:
    links = [line.strip() for line in f.readlines()] #strip() entfernt \n 

print(links[0])

https://finance.yahoo.com/news/australia-raise-fees-foreigners-buying-211338617.html


In [4]:
from bs4 import BeautifulSoup
import requests

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import time

import sqlite3

model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


for link in links:
    link = link.strip()
    res = requests.get(link)

    soup = BeautifulSoup(res.text, "html.parser")

    title = soup.find("title").text
    print(title)

    soup = soup.find("div", attrs={'class':'caas-body'})

    if(soup == None or title == 'Yahoo'):
        continue

    for div in soup.find_all('div'):
        div.unwrap()

    for ul in soup.find_all('ul'):
        # Finde das unmittelbar vorhergehende <p>-Tag, wenn vorhanden
        p_tag = ul.find_previous_sibling('p')

        # Wenn ein <p>-Tag gefunden wurde, entferne es
        if p_tag:
            p_tag.decompose()

    # Entferne alle <div>-Elemente
    for ul in soup.find_all("ul"):
        ul.decompose()

    for button in soup.find_all("button"):
        button.decompose()

    for p_tag in soup.find_all('p', string="©2023 Bloomberg L.P."):
        p_tag.decompose()

    # Dein Text
    text = ""

    for p in soup.find_all('p'):
        text = text + p.text + "\n"
    # Tokenisiere den Text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Wende das Modell an
    with torch.no_grad():
        outputs = model(**inputs)

    # Hole die Modellvorhersagen
    predictions = outputs.logits

    # Konvertiere Vorhersagen in Wahrscheinlichkeiten und hole das maximale
    predictions = torch.nn.functional.softmax(predictions, dim=1)
    predicted_class = torch.argmax(predictions).item()

    # Zuordnung der Klassen
    classes = ["bearish", "neutral", "bullish"]

    # Ergebnis
    print(f"Sentiment: {classes[predicted_class]}")
    # speichere das ergebnis in einer datenbank

    conn = sqlite3.connect('sentiment.db')
    c = conn.cursor()

    # Create table with title, link, text, sentiment
    c.execute('''CREATE TABLE IF NOT EXISTS sentiment
                (title text, link text, text text, sentiment text)''')

    # Insert a row of data
    c.execute("INSERT INTO sentiment VALUES (?, ?, ?, ?)", (title, link, text, classes[predicted_class]))
    
    conn.commit()
    conn.close()

Australia Will Raise Fees for Foreigners Buying Existing Houses
Sentiment: bullish
Global Rate-Cut Standoff Looms in 2023 Policy Finale
Sentiment: bearish
Adani Bond Rebound Helps Erase Short Seller Hit: Credit Weekly
Sentiment: bullish
UBS Taps JPMorgan’s Rueger as Co-Head of Equity Capital Markets
Sentiment: neutral
Still-Hot Jobs and a Medical Milestone: Saturday US Briefing
Sentiment: bullish
Why job candidates are 'ghosting' employers like never before
Sentiment: bullish
Decisive moment arrives with $4 trillion stocks rally at stake
Sentiment: bullish
QUOTES-EU clinches deal on landmark AI Act - reaction
Sentiment: bullish
Here's the right way to lend or gift money to help your loved one buy a home
Sentiment: bearish
Marex Shuns London in Favor of New York for Second IPO Attempt
Sentiment: bullish
QUOTES-EU clinches deal on landmark AI Act
Sentiment: bullish
EU clinches deal on landmark AI Act
Sentiment: bullish
Pessimistic consumers are suddenly feeling good
Sentiment: bullish
Oi